In [ ]:
%%capture

!pip install gdown gnss_lib_py

In [ ]:
%%capture

import gdown

FILE_URL = 'https://drive.google.com/uc?id=1BomwL31p7KvUz-UzQkR4tcB9Paq3IY0Y'
gdown.download(FILE_URL, output='gnss_log_2024_11_28_04_57_56.txt', quiet=True)

In [ ]:
import gnss_lib_py as glp

raw_data = glp.AndroidRawGnss(
  input_path="/content/gnss_log_2024_11_28_04_57_56.txt",
  filter_measurements=True,
  measurement_filters={"sv_time_uncertainty" : 500.},
  verbose=True,
)

In [ ]:
full_states = glp.add_sv_states(raw_data, source="precise", verbose=True)

In [ ]:
full_states["corr_pr_m"] = full_states["raw_pr_m"] \
                         + full_states['b_sv_m']

In [ ]:
full_states = full_states.where("gnss_id",("gps","galileo"))

In [ ]:
wls_estimate = glp.solve_wls(full_states)

In [ ]:
import numpy as np
wls_estimate = wls_estimate.where('x_rx_wls_m', np.nan, condition='neq')

In [ ]:
raw_fig = glp.plot_map(wls_estimate)
raw_fig.show()